[View in Colaboratory](https://colab.research.google.com/github/christianmerkwirth/colabs/blob/master/TraingCifar10FromColabAndInCloud.ipynb)

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
import os
os.chdir('models/tutorials/image/cifar10_estimator/')

First let's convert the training data to tfrecord format.

In [0]:
!python generate_cifar10_tfrecords.py --data-dir=${PWD}/cifar-10-data

In [0]:
!ls -lah ${PWD}/cifar-10-data

Run the model training locally (on the notebook instance). Use the python modules from the git repo.

In [0]:
!python cifar10_main.py --data-dir=${PWD}/cifar-10-data \
                       --job-dir=/tmp/cifar10 \
                       --num-gpus=1 \
                       --train-steps=2000

Authenticate to enable access to gcp.

In [0]:
from google.colab import auth
auth.authenticate_user()

Start the multigpu training. 

**WARNING. This training will run on GCP and will be billed against your account.**

In [0]:
%%bash

gcloud config set compute/region us-central1
gcloud config set project MYCOOLPROJECTNAME

# Make sure that the bucket is in the same region as the compute to allow fast
# data transfer.
MY_BUCKET=gs://MYCOOLBUCKETNAME
# Now we copy the training data from the notebook local storage to the cloud
# bucket.
gsutil cp -r ${PWD}/cifar-10-data $MY_BUCKET

# Move up one directory level since we need to have the code packaged into a
# package directory.
cd ..
gcloud ml-engine jobs submit training cifarmultigpu_20180806_01 \
    --runtime-version 1.4 \
    --job-dir=$MY_BUCKET/model_dirs/cifarmultigpu \
    --config cifar10_estimator/cmle_config.yaml \
    --package-path cifar10_estimator/ \
    --module-name cifar10_estimator.cifar10_main \
    -- \
    --data-dir=$MY_BUCKET/cifar-10-data \
    --num-gpus=4 \
    --train-steps=40000